# Developing an integration routine.

In this notebook, I develop a struct and methods that 
describe wiener processes, their refinement, and integration with respect to said wiener process.

I then use that to integrate a single stochastic path.

In [1]:
using Random, Plots
import Statistics
using LinearAlgebra

In [2]:
T = 1.0
N = 25000
Δt = T/N

4.0e-5

In [3]:
struct WienerProcess
    #the two key things to know are 
    path::Vector{Float64}
    Δt::Float64
   
    #This creates a wiener process using a given path, while 
    #enforcing the invariant that a WienerProcess starts at zero
    WienerProcess(p,d) = p[1] != 0.0 ? new(vcat(0.0,p),d) : new(p,d)
end

In [4]:
#Adding methods (not as member functions though)
function get_steps(p::WienerProcess)
    #This pulls the steps out of the WienerProcess
    len = length(p.path)
    if p.path[1] != 0.0
        error("test")
    end 
    steps = p.path[2:len] - p.path[1:len-1]
end

function refine(p::WienerProcess)
    #This refines the resolution of the given WienerProcess, and returns another in it's stead
    len = length(p.path)
    arr = zeros(2*len-1)
    
    for ind in 1:len-1
        arr[2*ind-1] = p.path[ind]
        arr[2*ind] =  (0.5 * (p.path[ind] + p.path[ind+1]) .+ (((√p.Δt)/2)*randn(1)))[1]
    end
    arr[2*len-1] = p.path[len]
    
    return WienerProcess(arr, p.Δt/2)
end

function spread_walk(walk)
    len = length(walk)
    arr = zeros(2*len-1)
    
    for ind in 1:len-1
        arr[2*ind-1] = walk[ind]
        arr[2*ind] =  (0.5 * (walk[ind] + walk[ind+1]) ) 
    end
    arr[2*len-1] = walk[len]
    
    return arr
end

spread_walk (generic function with 1 method)

In [5]:
#Generators
function WienerProcessFromSteps(steps::Vector{Float64}, domain_length::Float64)
    #this function builds a wiener process out of a given set of steps.
    #it uses the domain length to calculate Δt
    
    #TODO: implement checking for begining with zero
    return WienerProcess(steps,domain_length/length(steps))
end

function WienerProcess(size::Int, domain_length::Float64)
    #This function builds a wiener process out of randomly drawn steps
    #it uses the domain length to calculate Δt
    
    #TODO: implement checking for begining with zero
    return WienerProcess(cumsum(randn(size)),domain_length/size)
end

WienerProcess

In [6]:
function integrate(f::Function, wp::WienerProcess)
    #This function integrates across the time domain.
    
    sum = 0
    steps = get_steps(wp)
    
    for i in 1:length(wp.path)-1
        sum += f(wp.path[i]) * (steps[i]) 
    end
    
    return sum
end

integrate (generic function with 1 method)

In [7]:
#Generate a WienerProcess
tp = WienerProcess(N,T)

WienerProcess([0.0, 1.8829949734907265, 3.3250873836127597, 3.096714737918071, 4.438201856665624, 4.2705168154272615, 5.754823899159714, 6.465684995789774, 5.9714935176082395, 5.512718279617088  …  -18.226557280145187, -20.23683250735481, -20.649737497546198, -20.722837408881787, -22.180519087797773, -22.605585133344643, -23.41359522068531, -21.909119865011412, -22.170504831889502, -21.99102602124512], 4.0e-5)

In [8]:
#test functions
slf(x) = x
unit(x) = 1

unit (generic function with 1 method)

Integrate $f(W_t) = 1$

In [9]:
integrate(unit,tp),tp.path[length(tp.path)]

(-21.99102602124512, -21.99102602124512)

In [10]:
rtp = refine(tp)
integrate(unit,rtp),rtp.path[length(rtp.path)]

(-21.99102602124512, -21.99102602124512)

# Answering the question given
integrate $f(W_t) = W_t$, refining the partition as you go.

In [14]:
function time_refinements(tp, itt = 10)
    iterated_test = tp
    for iteration in 1:itt
        iterated_test = refine(iterated_test)

        integral = @time integrate(slf,iterated_test)

        path_length = length(iterated_test.path)
        las_value = iterated_test.path[path_length]
        symbolic_integral = 0.5 * (las_value^2 - T )

        println("iteration ",iteration, ". Number of discrete points ", path_length)
        println("\t",integral, " " , symbolic_integral)
        println("\tAbsolute Error: ",abs(integral - symbolic_integral))
        println("\tSquared Error: ",(integral - symbolic_integral)^2)
        println("\tPercentage Error: ",(integral - symbolic_integral)/symbolic_integral, "\n")
    end
end

time_refinements (generic function with 2 methods)

In [17]:
@time time_refinements(tp, 10)

  0.000418 seconds (6 allocations: 1.145 MiB)
iteration 1. Number of discrete points 50001
	-5967.987889600968 241.30261273353997
	Absolute Error: 6209.290502334507
	Squared Error: 3.855528854238152e7
	Percentage Error: -25.73237990253822

  0.000953 seconds (6 allocations: 2.289 MiB)
iteration 2. Number of discrete points 100001
	-2863.3424087299854 241.30261273353997
	Absolute Error: 3104.6450214635256
	Squared Error: 9.638820709298255e6
	Percentage Error: -12.866188999336906

  0.001841 seconds (6 allocations: 4.578 MiB)
iteration 3. Number of discrete points 200001
	-1311.0221622743804 241.30261273353997
	Absolute Error: 1552.3247750079204
	Squared Error: 2.4097122071033907e6
	Percentage Error: -6.433103883222704

  0.003813 seconds (6 allocations: 9.156 MiB)
iteration 4. Number of discrete points 400001
	-534.8595280068615 241.30261273353997
	Absolute Error: 776.1621407404015
	Squared Error: 602427.6687187228
	Percentage Error: -3.216550918980242

  0.007489 seconds (6 allocations